In [4]:
import pandas as pd 
from glob import glob
import os
import office365
import requests
import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings("ignore")

In [157]:
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
f = glob("Dthu\*csv")
files_Dthu_parquet = glob(os.path.join(path, "dthu_parquet_files\\*.parquet"))
files_CP = glob(os.path.join(path, "CP\\*.csv"))
files_Others = glob(os.path.join(path, "Others\\*.csv"))
files_SLBan = glob(os.path.join(path, "SLBan\\*.csv"))
files_TK = glob(os.path.join(path, "TK\\*.csv"))

# Cphi

In [55]:
df_cp = pd.read_parquet(r"https://raw.githubusercontent.com/Huynh-Tr/report/main/cphi.parquet")
# df_cp = df[df["Month year"] != "Month year"]
df_cp["Month year"] = pd.to_datetime(df_cp["Month year"]).dt.to_period('M')

# create MaCH column with condition right 4 chartacters of cost center code if not null else right 4 characters of GL Account Code
df_cp["MaCH"] = df_cp["Cost Center Code"].fillna(df_cp["Plant Code"].astype('str')).astype('int').astype('str').str[-4:] \
    .replace('0000', 'P015').replace('9999', 'P015')

df_cp["FM. Loc Amt"] = df_cp["FM. Loc Amt"].astype('float')

cols = ["Month year", "GL Account Code", "GL LV1 Name", "FM. Loc Amt", "PnL Name", "MaCH"]
gl_code_exl = 64110120 # list of GL Account Code
gl_code_range = [64100000, 64252100]

# filter data cols with gl_code_exl and in range gl_code_range
df_cp = df_cp[cols][(df_cp["GL Account Code"] != gl_code_exl) & (df_cp["GL Account Code"].between(gl_code_range[0], gl_code_range[1]))]

df_cp.head()

,Month year,GL Account Code,GL LV1 Name,FM. Loc Amt,PnL Name,MaCH
0,2022-01,64130110,CP CCDC,-552500.0,Chi phí bán hàng,1013
1,2022-01,64130140,CP CCDC,-4387500.0,Chi phí bán hàng,1013
2,2022-01,64140110,CP KH TSCĐ,-3336389.0,Chi phí bán hàng,1013
3,2022-01,64140130,CP KH TSCĐ,-1533889.0,Chi phí bán hàng,1013
4,2022-01,64110100,CP Nhân viên,-111131428.0,Chi phí bán hàng,1115


In [3]:
list(range(4, 13))

[4, 5, 6, 7, 8, 9, 10, 11, 12]

In [57]:
df_cp["GL LV1 Name"].unique()

array(['CP CCDC', 'CP KH TSCĐ', 'CP Nhân viên', 'CP vật liệu phụ',
       'CP Còn lại', 'CP Thuê ngoài', 'CP hành chính', 'CP chuyển tiền',
       'CP Marketing & Khuyến mãi', 'CP sửa chữa, bảo trì'], dtype=object)

In [74]:
df_cp = pd.concat([pd.read_csv(file, encoding='utf-8', sep=',', header=0) for file in files_CP])
df_cp["Month year"] = pd.to_datetime(df_cp["Month year"]).dt.to_period('M')

# create MaCH column with condition right 4 chartacters of cost center code if not null else right 4 characters of GL Account Code
df_cp["MaCH"] = df_cp["Cost Center Code"].fillna(df_cp["Plant Code"].astype('str')).astype('int').astype('str').str[-4:] \
    .replace('0000', 'P015').replace('9999', 'P015')


cols = ["Month year", "GL Account Code", "GL LV1 Name", "FM. Loc Amt", "PnL Name", "MaCH"]
gl_code_exl = 64110120 # list of GL Account Code
gl_code_range = [64100000, 64252100]

# filter data cols with gl_code_exl and in range gl_code_range
df_cp = df_cp[cols][(df_cp["GL Account Code"] != gl_code_exl) & (df_cp["GL Account Code"].between(gl_code_range[0], gl_code_range[1]))]

df_cp.head()

,Month year,GL Account Code,GL LV1 Name,FM. Loc Amt,PnL Name,MaCH
0,2022-01,64130110,CP CCDC,-552500.0,Chi phí bán hàng,1013
1,2022-01,64130140,CP CCDC,-4387500.0,Chi phí bán hàng,1013
2,2022-01,64140110,CP KH TSCĐ,-3336389.0,Chi phí bán hàng,1013
3,2022-01,64140130,CP KH TSCĐ,-1533889.0,Chi phí bán hàng,1013
4,2022-01,64110100,CP Nhân viên,-111131428.0,Chi phí bán hàng,1115


In [ ]:
# group by GL LV1 Name and Month year
df_cp[df_cp["Month year"]=="2025-01"].groupby(["GL LV1 Name", "Month year"])["FM. Loc Amt"].sum() / 1e9

GL LV1 Name                Month year
CP CCDC                    2025-01      -0.744826
CP Còn lại                 2025-01      -0.817176
CP KH TSCĐ                 2025-01      -0.211752
CP Marketing & Khuyến mãi  2025-01      -0.007248
CP Nhân viên               2025-01      -8.924364
CP Thuê ngoài              2025-01      -2.867169
CP chuyển tiền             2025-01      -0.529042
CP hành chính              2025-01      -0.442921
CP sửa chữa, bảo trì       2025-01      -1.238189
CP vật liệu phụ            2025-01      -0.493743
Name: FM. Loc Amt, dtype: float64

# Dthu

In [99]:
df = pd.read_csv(files_Dthu[-1], encoding='utf-8', sep=',', header=0)
df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
lv3 = ["CAO"]
lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
account_code = [51131000, 51132000]
df_tsv = df[~df["Product Group 2 Name"].isin(lv2)][~df["Product Group 3 Name"].isin(lv3)][~df["Product Group 4 Name"].isin(lv4)][~df["GL Account Code"].isin(account_code)]
df_vm = df[df["Product Group 4 Name"].isin(lv4)]

# Dthu, LG TSV
DT_TSV = f'{df_tsv["FM 01_Invoices Revenue"].sum() / 1e9 :.2f}'
LG_TSV = f'{df_tsv["FM 07_Gross Profit"].sum() / 1e9 :.2f}'
print(DT_TSV, LG_TSV)

# Dthu, LG VM
DT_VM = f'{df_vm["FM 01_Invoices Revenue"].sum() / 1e9 :.2f}'
LG_VM = f'{df_vm["FM 07_Gross Profit"].sum() / 1e9 :.2f}'
print(DT_VM, LG_VM)

213.28 50.16
25.34 2.46


In [86]:
col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]
df_tsv = df_tsv[col]
df_tsv.head()

,Product Group 2 Name,Product Group 3 Name,Product Group 4 Name,Month year,Plant Code,Channel Description,FM 01_Invoices Revenue,FM 07_Gross Profit
0,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1013,Retail,29536292.0,5316533.0
1,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1115,Retail,99000.0,17820.0
2,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1115,Retail,-99000.0,-17820.0
3,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1473,Retail,396000.0,71280.0
4,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,2025-01,1473,Retail,-396000.0,-71280.0


In [80]:
# extract year from series
y = pd.to_datetime(df_tsv["Month year"]).dt.year.unique()
y

# filter by year in list
df_tsv[df_tsv["Month year"].str.contains('|'.join(map(str, y)))]
# df_tsv["Month year"].dt.year()
# pd.to_datetime(df_tsv["Month year"].unique(), format='%m-%Y').dt.strftime('%Y')

,Product Group 2 Name,Product Group 3 Name,Product Group 4 Name,Month year,Plant Code,Channel Description,FM 01_Invoices Revenue,FM 07_Gross Profit
0,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1013,Retail,29536292.0,5316533.0
1,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1115,Retail,99000.0,17820.0
2,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1115,Retail,-99000.0,-17820.0
3,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1473,Retail,396000.0,71280.0
4,LOẠI KHÁC,MISCELLANEOUS,BÁCH HÓA TỔNG HỢP,01-2025,1473,Retail,-396000.0,-71280.0
...,...,...,...,...,...,...,...,...
3841,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1565,Online,313409.0,NaN
3842,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1575,Retail,318182.0,NaN
3843,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1634,Online,318182.0,NaN
3844,THAU / HỢP KIM,THAU / HỢP KIM,TRANG SỨC KHÔNG GẮN ĐÁ,01-2025,1634,Retail,318182.0,NaN


In [46]:
parquet_file = 'output.parquet'

df = pd.read_parquet(parquet_file)
df = df[df["Month year"] == "Jan 2025"]
lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
lv3 = ["CAO"]
lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
account_code = ['0051131000', '0051132000']
col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", \
    "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]

df_tsv = df.copy()
df_tsv["FM 01_Invoices Revenue"] = df_tsv["FM 01_Invoices Revenue"].astype('float')
df_tsv["FM 07_Gross Profit"] = df_tsv["FM 07_Gross Profit"].astype('float')

df_tsv = df_tsv[~df_tsv["Product Group 2 Name"].isin(lv2)]
print(len(df_tsv))
df_tsv = df_tsv[~df_tsv["Product Group 3 Name"].isin(lv3)]
df_tsv = df_tsv[~df_tsv["Product Group 4 Name"].isin(lv4)]
df_tsv = df_tsv[~df_tsv["GL Account Code"].isin(account_code)]

# print(df_tsv["GL Account Code"].unique())
# df_tsv[df_tsv["Product Group 4 Name"] == "KIM CƯƠNG"][["FM 01_Invoices Revenue", "FM 07_Gross Profit"]].sum() / 1e9
# df_tsv = df_tsv[col]
# print(df_tsv['FM 01_Invoices Revenue'].astype('float').sum() / 1e9)
df_tsv.groupby(by=['Product Group 4 Name'])[["FM 01_Invoices Revenue", "FM 07_Gross Profit"]].sum() / 1e9
df_tsv[["FM 01_Invoices Revenue", "FM 07_Gross Profit"]].sum() / 1e9
# df_tsv["FM 01_Invoices Revenue"].sum() / 1e9 / 2

7728


FM 01_Invoices Revenue    426.567428
FM 07_Gross Profit        100.327153
dtype: float64

In [ ]:
import pandas as pd 
from glob import glob
import os
import requests

import warnings
warnings.filterwarnings("ignore")

parquet = r"https://raw.githubusercontent.com/Huynh-Tr/report/main/dthu.parquet"

def dthu():
    df = pd.read_parquet(parquet)
    df = df[df["Month year"] != "Month year"]
    df["Month year"] = pd.to_datetime(df["Month year"]).dt.to_period('M')
    lv2 = ["DÂY ĐỒNG HỒ", "MẮT KÍNH", "ĐỒNG HỒ"]
    lv3 = ["CAO"]
    lv4 = ["TRANH/TƯỢNG/BIỂU TƯỢNG", "VÀNG ÉP SIÊU PNJ", "VÀNG ÉP SIÊU SJC"]
    account_code = [51131000, 51132000]
    col = ["Product Group 2 Name", "Product Group 3 Name", "Product Group 4 Name", "Month year", \
        "Plant Code", "Channel Description", "FM 01_Invoices Revenue", "FM 07_Gross Profit"]

    df_tsv = df.copy()
    df_tsv = df_tsv[~df_tsv["Product Group 2 Name"].isin(lv2)]
    df_tsv = df_tsv[~df_tsv["Product Group 3 Name"].isin(lv3)]
    df_tsv = df_tsv[~df_tsv["Product Group 4 Name"].isin(lv4)]
    df_tsv = df_tsv[~df_tsv["GL Account Code"].isin(account_code)]
    df_tsv = df_tsv[col]

    df_vm = df.copy()
    df_vm = df_vm[df_vm["Product Group 4 Name"].isin(lv4)]
    df_vm = df_vm[col]

    return df_tsv, df_vm

def chitieu(year, month, plant_code='Select All'):
    data_tsv, data_vm = dthu()
    if plant_code == 'Select All':
        sum_fm_tsv01 = data_tsv[data_tsv["Month year"].dt.year.isin(year)]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01 = data_vm[data_vm["Month year"].dt.year.isin(year)]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07 = data_tsv[data_tsv["Month year"].dt.year.isin(year)]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07 = data_vm[data_vm["Month year"].dt.year.isin(year)]["FM 07_Gross Profit"].sum() / 1e9
    else:
        sum_fm_tsv01 = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_vm01 = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.year.isin(year))]["FM 01_Invoices Revenue"].sum() / 1e9
        sum_fm_tsv07 = data_tsv[(data_tsv["Plant Code"].isin(plant_code)) & (data_tsv["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9
        sum_fm_vm07 = data_vm[(data_vm["Plant Code"].isin(plant_code)) & (data_vm["Month year"].dt.year.isin(year))]["FM 07_Gross Profit"].sum() / 1e9

    return sum_fm_tsv01, sum_fm_vm01, sum_fm_tsv07, sum_fm_vm07     
    

sum_fm_tsv01, sum_fm_vm01, sum_fm_tsv07, sum_fm_vm07 = chitieu([2025], 1, plant_code='Select All')

NameError: name 'data_tsv' is not defined

# Parquet

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from glob import glob  # For handling multiple CSV files
import os

In [156]:
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"

csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory
# create a directory to save parquet files
if not os.path.exists(os.path.join(path, ".\parquet_files")):
    os.makedirs(os.path.join(path, ".\parquet_files"))

parquet_directory = os.path.join(path, "dthu_parquet_files")

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    table = pa.Table.from_pandas(df)

    # Create parquet filename based on csv filename:
    parquet_file = parquet_directory + "\\" + os.path.basename(csv_file).replace('.csv', '.parquet')

    pq.write_table(table, parquet_file)


In [ ]:


# if not os.path.exists(os.path.join(path, ".\parquet_files")):
#     os.makedirs(os.path.join(path, ".\parquet_files"))

# parquet_directory = os.path.join(path, "parquet_files\\file.parquet")

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from glob import glob
import os

def csv_to_parquet_append_legacy(csv_files, parquet_file):  # Legacy method
     if not os.path.exists(parquet_file):
         try:
             first_df = pd.read_csv(csv_files)
             table = pa.Table.from_pandas(first_df)
             pq.write_table(table, parquet_file)
         except FileNotFoundError:
             print(f"Error: First CSV file '{csv_files}' not found. Cannot create Parquet file.")
             return
         except Exception as e:
             print(f"Error creating initial Parquet file: {e}")
             return

     for csv_file in csv_files:
         try:
             df = pd.read_csv(csv_file)
             table = pa.Table.from_pandas(df)

             if os.path.exists(parquet_file):  # Read existing Parquet file
                 existing_table = pq.read_table(parquet_file)
                 combined_table = pa.concat_tables([existing_table, table]) # Concatenate tables
                 pq.write_table(combined_table, parquet_file) # Write combined table back

             else: #If file doesn't exist, write the table directly
                 pq.write_table(table, parquet_file)

         except FileNotFoundError:
             print(f"Error: CSV file '{csv_file}' not found.")
         except pa.lib.ArrowInvalid as e:
             print(f"Error converting CSV to Parquet: {e}. Check data types for file: {csv_file}")
         except Exception as e:
             print(f"An unexpected error occurred while processing {csv_file}: {e}")

# Example usage:
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory

parquet_file = 'output.parquet'  # Replace with your desired Parquet file path

csv_to_parquet_append(csv_files, parquet_file)

print(f"CSV files appended to Parquet file: {parquet_file}")

<>:48: SyntaxWarning: invalid escape sequence '\p'
<>:48: SyntaxWarning: invalid escape sequence '\p'
C:\Users\huynh.th\AppData\Local\Temp\ipykernel_32680\2253336765.py:48: SyntaxWarning: invalid escape sequence '\p'
  path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"


An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202101.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202102.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202103.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202104.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202105.csv: __cinit__() got an unexpected keyword argument 'append'
An unexpected error occurred while processing D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit\Dthu\202106.csv: __cinit__() got an unexpected 

In [13]:
# create an empty parquet file
import pyarrow as pa
import pyarrow.parquet as pq

# Create an Arrow table with an empty schema
parquet_file = 'output.parquet'
table = pa.Table.from_pandas(pd.DataFrame())
pq.write_table(table, parquet_file)

In [47]:
import pyarrow as pa
import pyarrow.parquet as pq

# Create an Arrow table with an empty schema
parquet_file = 'output.parquet'
table = pa.Table.from_pandas(pd.DataFrame())
# pq.write_table(table, parquet_file)

# Path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
path = "D:\pnj.com.vn\HuynhTN - Documents\Project\streamlit"
csv_files = glob(os.path.join(path, "Dthu\\*.csv"))  # Get all CSV files in the directory

pd_df = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files])
table = pa.Table.from_pandas(pd_df)
pq.write_table(table, parquet_file)